In [ ]:
!pip install -q transformers accelerate bitsandbytes gradio torch pillow

import torch
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import gradio as gr
from PIL import Image
import re
from typing import List, Tuple

class RiverPollutionAnalyzer:
    def __init__(self):
        # Initialize model with 4-bit quantization
        self.processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")
        self.model = InstructBlipForConditionalGeneration.from_pretrained(
            "Salesforce/instructblip-vicuna-7b",
            device_map="auto",
            torch_dtype=torch.float16,
            load_in_4bit=True
        )

        self.pollutants = [
            "plastic waste", "chemical foam", "industrial discharge",
            "sewage water", "oil spill", "organic debris",
            "construction waste", "medical waste", "floating trash",
            "algal bloom", "toxic sludge", "agricultural runoff"
        ]

        self.severity_descriptions = {
            1: "Minimal pollution - Slightly noticeable",
            2: "Minor pollution - Small amounts visible",
            3: "Moderate pollution - Clearly visible",
            4: "Significant pollution - Affecting water quality",
            5: "Heavy pollution - Obvious environmental impact",
            6: "Severe pollution - Large accumulation",
            7: "Very severe pollution - Major ecosystem impact",
            8: "Extreme pollution - Dangerous levels",
            9: "Critical pollution - Immediate action needed",
            10: "Disaster level - Ecological catastrophe"
        }

    def analyze_image(self, image):
        """Analyze river pollution with robust parsing"""
        if not isinstance(image, Image.Image):
            image = Image.fromarray(image)

        prompt = """Analyze this river pollution scene and provide:
1. List ALL visible pollutants ONLY from: [plastic waste, chemical foam, industrial discharge, sewage water, oil spill, organic debris, construction waste, medical waste, floating trash, algal bloom, toxic sludge, agricultural runoff]
2. Estimate pollution severity from 1-10

Respond EXACTLY in this format:
Pollutants: [comma separated list]
Severity: [number]"""

        inputs = self.processor(
            images=image,
            text=prompt,
            return_tensors="pt"
        ).to("cuda", torch.float16)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=200,
                temperature=0.5,
                top_p=0.85,
                do_sample=True
            )

        analysis = self.processor.batch_decode(outputs, skip_special_tokens=True)[0]
        pollutants, severity = self._parse_response(analysis)
        return self._format_analysis(pollutants, severity)

    def _parse_response(self, analysis: str) -> Tuple[List[str], int]:
        """Robust parsing of model response"""
        pollutants = []
        severity = 3

        # Extract pollutants
        pollutant_match = re.search(
            r'(?i)(pollutants?|contaminants?)[:\s]*\[?(.*?)(?:\]|Severity|severity|$)',
            analysis
        )

        if pollutant_match:
            pollutants_str = pollutant_match.group(2).strip()
            pollutants = [
                p.strip().lower()
                for p in re.split(r'[,;]|\band\b', pollutants_str)
                if p.strip().lower() in self.pollutants
            ]

        # Extract severity
        severity_match = re.search(
            r'(?i)(severity|level)[:\s]*(\d{1,2})',
            analysis
        )

        if severity_match:
            try:
                severity = min(max(int(severity_match.group(2)), 1), 10)
            except:
                severity = self._calculate_severity(pollutants)
        else:
            severity = self._calculate_severity(pollutants)

        return pollutants, severity

    def _calculate_severity(self, pollutants: List[str]) -> int:
        """Weighted severity calculation"""
        if not pollutants:
            return 1

        weights = {
            "medical waste": 3, "toxic sludge": 3, "oil spill": 2.5,
            "chemical foam": 2, "industrial discharge": 2, "sewage water": 2,
            "plastic waste": 1.5, "construction waste": 1.5, "algal bloom": 1.5,
            "agricultural runoff": 1.5, "floating trash": 1, "organic debris": 1
        }

        avg_weight = sum(weights.get(p, 1) for p in pollutants) / len(pollutants)
        return min(10, max(1, round(avg_weight * 3)))

    def _format_analysis(self, pollutants: List[str], severity: int) -> str:
        """Generate formatted report"""
        severity_bar = f"""📊 Severity: {severity}/10
{"█" * severity}{"░" * (10 - severity)}
{self.severity_descriptions.get(severity, '')}"""

        pollutants_list = "\n🔍 No pollutants detected" if not pollutants else "\n".join(
            f"{i}. {p.capitalize()}" for i, p in enumerate(pollutants[:5], 1))

        return f"""🌊 River Pollution Analysis 🌊
{pollutants_list}
{severity_bar}"""

# Initialize analyzer
analyzer = RiverPollutionAnalyzer()

import gradio as gr

 # Import your actual analyzer

css = """
/* (Keep all your CSS styles) */
"""

with gr.Blocks(theme=gr.themes.Soft(), css=css) as demo:
    with gr.Column(elem_classes="header"):
        gr.Markdown("# 🌍 River Pollution Analyzer")
        gr.Markdown("### AI-powered water pollution detection")

    with gr.Row(elem_classes="side-by-side"):
        # Left Panel
        with gr.Column(elem_classes="left-panel"):
            with gr.Group():
                image_input = gr.Image(type="pil", label="Upload River Image", height=300)
                analyze_btn = gr.Button("🔍 Analyze Pollution", variant="primary")

            with gr.Group(elem_classes="analysis-box"):
                gr.Markdown("### 📊 Analysis report")
                analysis_output = gr.Markdown()

        # Right Panel
        with gr.Column(elem_classes="right-panel"):
            with gr.Group(elem_classes="chat-container"):
                chatbot = gr.Chatbot(label="Pollution Analysis Q&A", height=400)
                with gr.Row():
                    chat_input = gr.Textbox(placeholder="Ask about pollution sources...",
                                         label="Your Question", container=False, scale=5)
                    chat_btn = gr.Button("💬 Ask", variant="secondary", scale=1)
                clear_btn = gr.Button("🧹 Clear Chat History", size="sm")

    # Connect to your actual analyzer functions
    analyze_btn.click(
        analyzer.analyze_image,
        inputs=image_input,
        outputs=analysis_output
    )

    chat_input.submit(
        lambda msg, chat: ("", chat + [(msg, analyzer.analyze_chat(msg))]),
        inputs=[chat_input, chatbot],
        outputs=[chat_input, chatbot]
    )

    chat_btn.click(
        lambda msg, chat: ("", chat + [(msg, analyzer.analyze_chat(msg))]),
        inputs=[chat_input, chatbot],
        outputs=[chat_input, chatbot]
    )

    clear_btn.click(
        lambda: None,
        outputs=[chatbot]
    )

    # Examples using your real analyzer
    gr.Examples(
        examples=[
            ["https://huggingface.co/spaces/atharwaah1work/tarak.AI/resolve/main/polluted_river1.jpg"],
            ["https://drive.google.com/uc?export=view&id=1WGcXwFhpbD1LrtbQ8E5IZZN3nEGfcwuN"]
        ],
        inputs=image_input,
        outputs=analysis_output,
        fn=analyzer.analyze_image,
        cache_examples=True,
        label="Try example images:"
    )

demo.launch()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<ipython-input-4-b3b09572975b>:166: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Pollution Analysis Chat", height=400)


Caching examples at: '/content/.gradio/cached_examples/39'
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fe6f9a6454890ad3b1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
